In [61]:
import numpy as np
#from imageio import imread, imwrite
import cv2
# from matplotlib,pyplot as plt # not working for me

## Naive Method: Increase blue channel

In [62]:
img = cv2.imread('grainger.jpg')

arr = img * np.array([0.5, 0.2, 0.1])
arr2 = (255 * arr/arr.max()).astype(np.uint8)

gamma = 2
gamma_img = np.array(255 * (arr2/255) ** gamma, dtype="uint8")
cv2.imwrite("night_final.png", gamma_img)

True

## Advanced Method: Remove illumination, decrease exposure, relight with night illuminants, and add noise

### Remove illumination

Stacked RGGB Bayer image:
$$ \mathbf{I}_{day} \in R^{\frac{H}{2} \times \frac{W}{2} \times 4} $$

In [63]:
img = cv2.imread('grainger.jpg')
# I_day = img # replace with raw format

# The process begins with a "minimally processed Bayer image recorded by the camera sensor".
# We could synthesize the effect of a raw Bayer image with the code below,
# then interpolate the pixel values (or downsample, but I suppose it's arbitrary). This is why I think their I_day is H/2 x W/2
# OR this may all be useless and we should just contain with the normal rgb image turned into rggb

# Created with the help of ChatGPT.
def apply_bayer_pattern(rgb_image):
    print(rgb_image.shape)
    # Create a blank Bayer pattern image with the same dimensions as the RGB image
    bayer_image = np.zeros(rgb_image.shape)

    # Apply the BGGR Bayer pattern to the image
    bayer_image[::2, ::2, 0] = rgb_image[::2, ::2, 0]  # Blue pixels
    bayer_image[1::2, ::2, 1] = rgb_image[1::2, ::2, 1]  # Green pixels in even rows
    bayer_image[::2, 1::2, 1] = rgb_image[::2, 1::2, 1]  # Green pixels in odd rows
    bayer_image[1::2, 1::2, 2] = rgb_image[1::2, 1::2, 2]  # Red pixels

    return bayer_image

I_day = apply_bayer_pattern(img)


(1200, 1800, 3)


Normalized image:
$$ \mathbf{I}_n = (\mathbf{I}_{day}-b_l) / (w_l-b_l) $$

In [64]:
print(I_day.shape)
# wl = np.mean(np.max(img, axis=(0,1)))
# bl = np.mean(np.min(img, axis=(0,1)))
wl = np.max(I_day)
bl = np.min(I_day)

I_n = (I_day - bl) / (wl - bl)
cv2.imwrite('bayer_image_I_n.jpg', I_n * 255)


(1200, 1800, 3)


True

White-balanced image:
$$ \mathbf{I}_w = \mathbf{I}_n\mathbf{L}_{day} $$
where
$$ {L}_{day} = diag(\frac{1}{r}, \frac{1}{g}, \frac{1}{g}, \frac{1}{b}) $$

In [65]:
print(I_n.shape)

for c in range(I_n.shape[-1]):
    pass

# https://chat.openai.com/share/f90475c2-2156-42c5-afba-2b8a22fee9f1

L_day = np.identity(I_n.shape[-1])
I_w = I_n @ L_day

# We might just have to come up with some values for r, g, b to be the average shift of intensities to color balance to neutral
# Or create a simple algorithm

(1200, 1800, 3)


### Lowering brightness

`D`: dictionary for the normalized mean intensity value of each Bayer image
(could just make list with index as key for the image)

In [66]:
# I believe bayer_images should have a few nighttime images, which would make D have normalized means of random nighttime photos
bayer_images = np.array([I_day])
print(bayer_images.shape)

# contains normalized mean intensity values of each Bayer image
D = np.array([np.mean(img) / np.max(img) for img in bayer_images])

global_scale_factor_d = np.random.choice(D)
print(global_scale_factor_d)

I_e = I_w * global_scale_factor_d


(1, 1200, 1800, 3)
0.19552784495279593


### Adding illuminants

`L`: dictionary for nighttime illuminants

In [67]:
# Make a few samples of nighttime illuminants

# Number of nighttime illuminants
M = 5

# Create 5 dummy illuminants
illuminants = np.random.rand(M * np.prod(I_e.shape)).reshape((M,) + I_e.shape)

chromaticity_pairs = []
for illum in illuminants:
    # Joint chromaticity values, normalized
    rg = illum[:,:,0] / illum[:,:,1]
    rg = (rg - np.min(rg)) / np.max(rg)
    gb = illum[:,:,-2] / illum[:,:,-1]
    gb = (gb - np.min(gb)) / np.max(gb)

    # chromaticity_pairs.append((rg, gb))
    chromaticity_pairs.append((np.mean(rg), np.mean(gb)))

chromaticity_pairs = np.array(chromaticity_pairs)
print(chromaticity_pairs.shape)

# Mean of normalized chromaticity values in L
# mu = np.mean(chromaticity_pairs, axis=(0, 2, 3))
mu = np.mean(chromaticity_pairs, axis=0)
print(mu, mu.shape)

# Covariance of normalized chromaticity values in L
# sigma = np.cov()
sigma = np.zeros((mu.shape[0], mu.shape[0]))
for pair in chromaticity_pairs:
    sigma += np.outer((pair - mu).T, pair - mu)
sigma /= M
print(sigma, sigma.shape)

(5, 2)
[1.83154188e-05 5.10948461e-06] (2,)
[[ 1.01181499e-10 -3.27515364e-11]
 [-3.27515364e-11  1.64122274e-11]] (2, 2)


### Adding noise

### Final Result